In [2]:
import pandas as pd
import time
import re
from fake_useragent import UserAgent
from urllib.request import Request, urlopen 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm

In [42]:
df = pd.read_excel('data/episode_score.xlsx', index_col=0)
df.tail()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,...,season_8,season_9,season_10,season_11,season_12,season_13,season_14,season_15,season_16,season_17
1085,달러,Dollar,2019,1,41min,"액션, 드라마, 로맨스",15,레바논,Dollar (TV Series 2019),https://www.imdb.com/title/tt10687564/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1086,러브 나우,真愛趁現在,2012,1,45min,"드라마, 코미디",15,대만,Love Now (TV Series 2012–2013),https://www.imdb.com/title/tt6273116/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1087,탕탕 라이브,糖糖Online,2019,1,24min,드라마,15+,대만,Tang tang online (TV Series 2019),https://www.imdb.com/title/tt11330500/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088,대송궁사,大宋宫词,2021,1,45min,드라마,15+,China,Palace of Devotion (TV Series 2021),https://www.imdb.com/title/tt10742204/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1089,"엘리트들, 못다 한 이야기: 나디아 구스만",Elite Histórias Breves: Nadia Guzmán,2021,1,11min,"드라마, 로맨스",NaN,스페인,Elite Short Stories: Nadia Guzmán (TV Mini Ser...,https://www.imdb.com/title/tt14671790/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 페이지의 각 요소 확인

In [11]:
df.loc[1,'IMDb_URL'] + 'episodes'

'https://www.imdb.com/title/tt7605396/episodes/'

In [10]:
# 에피스드 주소로 접속

driver = webdriver.Chrome()
driver.get(df.loc[6, 'IMDb_URL'] + 'episodes')

In [6]:
# 시즌 갯수 추출

season = driver.find_elements(By.CSS_SELECTOR,'a[data-testid="tab-season-entry"]')
season

[<selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd1d57924", element="f.D204E52448789B0817932019448DF653.d.50A6680FCE70663BBF3A7C4C1D21FFF2.e.79")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd1d57924", element="f.D204E52448789B0817932019448DF653.d.50A6680FCE70663BBF3A7C4C1D21FFF2.e.80")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd1d57924", element="f.D204E52448789B0817932019448DF653.d.50A6680FCE70663BBF3A7C4C1D21FFF2.e.81")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd1d57924", element="f.D204E52448789B0817932019448DF653.d.50A6680FCE70663BBF3A7C4C1D21FFF2.e.82")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd1d57924", element="f.D204E52448789B0817932019448DF653.d.50A6680FCE70663BBF3A7C4C1D21FFF2.e.83")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd

In [7]:
len(season)

7

In [5]:
episode_score = driver.find_elements(By.CSS_SELECTOR, '#__next > main > div > section > div > section > div > div > section:nth-child(2) > section > article')
episode_score

[<selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd1d57924", element="f.D204E52448789B0817932019448DF653.d.50A6680FCE70663BBF3A7C4C1D21FFF2.e.62")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd1d57924", element="f.D204E52448789B0817932019448DF653.d.50A6680FCE70663BBF3A7C4C1D21FFF2.e.59")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd1d57924", element="f.D204E52448789B0817932019448DF653.d.50A6680FCE70663BBF3A7C4C1D21FFF2.e.63")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd1d57924", element="f.D204E52448789B0817932019448DF653.d.50A6680FCE70663BBF3A7C4C1D21FFF2.e.64")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd1d57924", element="f.D204E52448789B0817932019448DF653.d.50A6680FCE70663BBF3A7C4C1D21FFF2.e.65")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09685fb7db23cc8f4136379dd

In [109]:
text = episode_score[0].text.split('\n')
text

['TOP-RATED',
 'S1.E1 ∙ Mugunghwa kkoch-i pideon nal',
 'Fri, Sep 17, 2021',
 'Hoping to win easy money, a broke and desperate Gi-hun agrees to take part in an enigmatic game. Not long into the first round, unforeseen horrors unfold.',
 '8.2',
 '/10',
 ' (16K)',
 'Rate']

In [119]:
# 시즌 정보 추출

for n in text:
    if n[0] == 'S':
        print(n[:5])

S1.E1


In [136]:
i = 0

for n in episode_score[-1].text.split('\n'):
    if n[0] == 'S':
        season_num = n[:5]
        year = episode_score[-1].text.split('\n')[i+1][-4:]
    i+=1

S1.E9
2021


In [126]:
# 평점과 투표자 수 추출

text[-4], text[-2].strip()[1:-1]

('8.2', '16K')

In [ ]:
# 24년 개봉 작품 확인

driver.find_elements(By.CSS_SELECTOR, 'article > div > div > div.sc-ccd6e31b-4.eMYVLm > div.sc-ccd6e31b-5.cXcoHr > span')

In [50]:
driver.find_elements(By.CSS_SELECTOR, 'section:nth-child(2) > section > article > div > div > div > div > div > span')

[<selenium.webdriver.remote.webelement.WebElement (session="1b50df5de6e8f5778d2ace8ef0c19414", element="f.0B75F2AD272C091E432BFE6A9C172A84.d.B6592C3750F0CB1342352268DB4B89C5.e.7921")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b50df5de6e8f5778d2ace8ef0c19414", element="f.0B75F2AD272C091E432BFE6A9C172A84.d.B6592C3750F0CB1342352268DB4B89C5.e.7861")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b50df5de6e8f5778d2ace8ef0c19414", element="f.0B75F2AD272C091E432BFE6A9C172A84.d.B6592C3750F0CB1342352268DB4B89C5.e.7801")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b50df5de6e8f5778d2ace8ef0c19414", element="f.0B75F2AD272C091E432BFE6A9C172A84.d.B6592C3750F0CB1342352268DB4B89C5.e.7741")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b50df5de6e8f5778d2ace8ef0c19414", element="f.0B75F2AD272C091E432BFE6A9C172A84.d.B6592C3750F0CB1342352268DB4B89C5.e.7681")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b50df5de6e8f57

In [52]:
score = driver.find_elements(By.CSS_SELECTOR, 'section:nth-child(2) > section > article > div > div > div > div > div > span')[0].text
score

'8.2\n/10\n (16K)'

In [64]:
# 평점 추출

score.split('\n')[0]

'8.2'

In [62]:
# 투표수 추출

score.split('\n')[2].strip()[1:-1]

'16K'

In [76]:
driver.get(df.loc[1,'IMDb_URL'] + 'episodes')

In [77]:
df.loc[1,'IMDb_URL'] + 'episodes'

'https://www.imdb.com/title/tt7605396/episodes'

In [40]:
columns = ['season_' + str(i) for i in range(1, 18)]
columns

['season_1',
 'season_2',
 'season_3',
 'season_4',
 'season_5',
 'season_6',
 'season_7',
 'season_8',
 'season_9',
 'season_10',
 'season_11',
 'season_12',
 'season_13',
 'season_14',
 'season_15',
 'season_16',
 'season_17']

In [41]:
df.loc[:,columns] = None
df.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,...,season_8,season_9,season_10,season_11,season_12,season_13,season_14,season_15,season_16,season_17
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,...,None,None,None,None,None,None,None,None,None,None
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,...,None,None,None,None,None,None,None,None,None,None
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,...,None,None,None,None,None,None,None,None,None,None
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,...,None,None,None,None,None,None,None,None,None,None
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,...,None,None,None,None,None,None,None,None,None,None


### 크롤링

In [17]:
# 스크롤을 위한 액션체인

action = ActionChains(driver)

In [384]:
driver.close()

In [28]:

for idx, url in tqdm(df.iterrows()):
    if type(url['IMDb_URL']) == float:
        continue
    else:
        driver.get(url['IMDb_URL'] + 'episodes')

    # 시즌 개수 추출
    season = driver.find_elements(By.CSS_SELECTOR,'a[data-testid="tab-season-entry"]')

    for s in range(len(season)):
        time.sleep(3)
        
        years = []
        score_dict = {}

        # 에피소드별 정보 추출
        episode_score = driver.find_elements(By.CSS_SELECTOR, '#__next > main > div > section > div > section > div > div > section:nth-child(2) > section > article')

        # 해당 시즌 에피소드의 연도만 추출
        for i in episode_score:
            y = 0
            for j in re.split('(\d\d\d\d)', i.text):
                if len(j) == 4:
                    years.append(j)
                    break
        
        if '2024' in years:
            break
        
        for row in episode_score:
            data = row.text
            data = data.split('\n')

            for n in data:
                if n[0] == 'S':
                    season_episode_num = n.split(' ∙ ')[0]
                    break
            
            try:
                if data[-1] == 'Watch options':
                    score = data[-5]
                    vote_num = data[-3].strip()[1:-1]
                    if len(score) > 3:
                        column = 'season_' + str(s + 1)
                        df.at[idx, column] = '평점 없음'
                        break
                    score_dict[season_episode_num] = [score, vote_num]
                else:
                    score = data[-4]
                    vote_num = data[-2].strip()[1:-1]
                    if len(score) > 3:
                        column = 'season_' + str(s + 1)
                        df.at[idx, column] = '평점 없음'
                        break
                    score_dict[season_episode_num] = [score, vote_num]
            except:
                break
        
        if len(score_dict.values()) == 0:
            continue
        else:
            column = 'season_' + str(s + 1)
            df.at[idx, column] = score_dict

        if len(df.at[idx, column]) == 1:
            df.at[idx, column] = '확인 필요'

        if len(score) > 3:
            df.at[idx, column] = '평점 없음'

        episode_score = 0

        if s < len(season) - 1:
            some_tag = driver.find_element(By.CSS_SELECTOR, '#next-season-btn > svg')
            action.move_to_element(some_tag).perform()
            driver.find_element(By.CSS_SELECTOR, '#next-season-btn > svg').click()

2it [00:12,  6.38s/it]


In [40]:
# 확인 필요 컬럼 추출

for idx, row in df.loc[:,'season_1':'season_17'].iterrows():
    for i in row:
        if i == '확인 필요':
            print(idx)

In [41]:
# 시즌 넘버가 잘못 지정된 밸류 추출

for idx, row in df.loc[:,'season_1':'season_17'].iterrows():
    s = 1
    for j in row:
        if type(j) == float:
            break
        elif j == None:
            continue
        elif type(j) != str:
            for i in j.keys():
                if 'S'+str(s) == i.split('.')[0]:
                    continue
                elif 'S'+str(s) != i.split('.')[0]:
                    print(idx)
                    break
        s+=1

In [75]:
# 문자열이 들어간 밸류 추출

for idx, row in df.loc[:,'season_1':'season_17'].iterrows():
    for j in row:
        if type(j) == float:
            break
        if type(j) != str:
            for i in j.values():
                if len(i[0]) > 3:
                    print(idx)

AttributeError: 'NoneType' object has no attribute 'values'

In [ ]:
# 뷰티풀숩을 이용한 크롤링

for idx, url in tqdm(df.loc[20:21].iterrows()):
    ua = UserAgent()
    req = Request(url['IMDb_URL'] + 'episodes', headers={"user-agent": ua.ie})
    html = urlopen(req)
    time.sleep(2)
    season_number = 1

    # 시즌 개수 추출
    season = driver.find_elements(By.CSS_SELECTOR,'a[data-testid="tab-season-entry"]')

    for i in range(len(season)):
        time.sleep(2)
        years = []
        score_dict = {}

        # 에피소드별 정보 추출
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        episode_score = soup.select('.sc-282bae8e-1.dSEzwa.episode-item-wrapper')

        # 해당 시즌 에피소드의 연도만 추출
        for i in episode_score:
            for j in re.split('(\d\d\d\d)', i.text):
                if len(j) == 4:
                    years.append(j)
        
        if '2024' in years:
            break

        for row in episode_score:
            
            try:
                # 시즌 번호와 에피소드 번호 추출
                season_episode_num = re.split('(S\d+.E\d+)', row.text.strip())[1]

                # 평점 추출
                score = re.split('.(\d\.?\d?)/10', row.text)[1]

                vote_num = re.split('(\d+\S?)', row.text)[-2]

                score_dict[season_episode_num] = [score, vote_num]
            except:
                pass
        
        if len(score_dict.values()) == 0:
            continue
        else:
            column = 'season_' + str(season_number)
            season_number += 1
            df.at[idx, column] = score_dict

        if len(df.at[idx, column]) == 1:
            df.at[idx, column] = '확인 필요'

        try:
            some_tag = driver.find_element(By.CSS_SELECTOR, '#next-season-btn > svg')
            action.move_to_element(some_tag).perform()
            driver.find_element(By.CSS_SELECTOR, '#next-season-btn > svg').click()
        except:
            pass

### 데이터 저장

In [30]:
# 시즌 내 에피스드 중 평점이 하나라도 없으면 평점 없음으로 처리
# 에피소드 중 2024년 중 공개한 작품이 포함되어 있을 경우 표기 없이 시즌 채로 제외. 시즌1부터 2024년 공개작 포함일 경우에만 '2024년 공개작 포함' 표시
# 에피소드 중 하나라도 평점이 없는 경우 시즌 채로 '평점 없음' 표시
# 에피소드 정보가 없을 경우 '에피소드 정보 없음' 표시
# 시즌 넘버가 붙지 않은 외전 시즌은 IMDb에서 붙여놓은 시즌 네임을 키 값으로 설정 (19, 149, 208, 331, 791)

df.to_excel('data/episode_score.xlsx')